# PYSPARK'DA NULL DEĞERLERİN YÖNETİLMESİ
Bu yazımızda Pyspark'da veri setlerinde karşımıza çıkabilen null değerler nasıl yönetilir ile aşağıda ki sorulara cevap arayacağız ;<br><br>

-Satır satır veriyi inceleyip hiç null değeri olmayan satırlar nasıl gösterilir?<br><br>
-Satır satır veriyi inceleyip n ve n'den fazla null değeri olmayan satırlar nasıl gösterilir?<br><br>
-İstenilen bir kolon için null değerler/null olmayan değerler nasıl filtrelenir?<br><br>
-Null değerleri herhangi bir numerik değerle (eğer kolon numerik ise) veya string bir değer ile (eğer ki kolon string ise)  değer ile nasıl değiştirilir?<br><br>
-Null değerler ortalama ve medyan değerleri ile nasıl değiştirilir?<br><br>
-Null değerler, avg ve count hesaplamalarını nasıl etkiler? Averaj hesabında null değerleri hesaba katılır mı?


In [1]:
##Spark session oluşturulur
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

Data setimizi manual oluşturalım. İki şekilde oluşturabiliriz. Birincisi StructType kullanarak ikincisi klasik şekilde.


In [2]:
##true for nullable, and false for not nullable
from pyspark.sql.types import *
schema = StructType([StructField('Id',IntegerType(), True),
                     StructField('Name',StringType(), True),
                     StructField('Height', DoubleType(),True),
                     StructField('Age', IntegerType(),True),
                     StructField('Gender', StringType(),True),
                     StructField('Profession', StringType(),True)])
 

In [3]:
df = spark.createDataFrame([(1, 'John', 1.79, 28,'M', 'Doctor'),
                            (2, 'Steve', 1.78, 45,'M', None),
                            (3, 'Emma', 1.75, None, None, None),
                            (4, 'Ashley',1.6, 33,'F', 'Analyst'),
                            (5, 'Olivia', 1.8, 54,'F', 'Teacher'),
                            (6, 'Hannah', 1.82, None, 'F', None),
                            (7, 'William', 1.7, 42,'M', 'Engineer'),
                            (None,None,None,None,None,None),
                            (8,'Ethan',1.55,38,'M','Doctor'),
                            (9,'Hannah',1.65,None,'F','Doctor'),
                            (10,'Xavier',1.64,43,None,'Doctor'),
                            (11,None,None,None,None,None)],schema)

Aşağıda klasik şekilde oluşturulan yol mevcut.


In [34]:
##
df = spark.createDataFrame([(1, 'John', 1.79, 28,'M', 'Doctor'),
                            (2, 'Steve', 1.78, 45,'M', None),
                            (3, 'Emma', 1.75, None, None, None),
                            (4, 'Ashley',1.6, 33,'F', 'Analyst'),
                            (5, 'Olivia', 1.8, 54,'F', 'Teacher'),
                            (6, 'Hannah', 1.82, None, 'F', None),
                            (7, 'William', 1.7, 42,'M', 'Engineer'),
                            (None,None,None,None,None,None),
                            (8,'Ethan',1.55,38,'M','Doctor'),
                            (9,'Hannah',1.65,None,'F','Doctor'),
                            (10,'Xavier',1.64,43,None,'Doctor'),
                            (11,None,None,None,None,None)]
                           , ['Id', 'Name', 'Height', 'Age', 'Gender', 'Profession'])

data type'ı ve kolonları görelim


In [5]:
print(df)

DataFrame[Id: bigint, Name: string, Height: double, Age: bigint, Gender: string, Profession: string]


Kolonları ve tiplerini görelim


In [6]:
df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Age: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Profession: string (nullable = true)



dataframe'i görelim


In [7]:
df.show()

+----+-------+------+----+------+----------+
|  Id|   Name|Height| Age|Gender|Profession|
+----+-------+------+----+------+----------+
|   1|   John|  1.79|  28|     M|    Doctor|
|   2|  Steve|  1.78|  45|     M|      null|
|   3|   Emma|  1.75|null|  null|      null|
|   4| Ashley|   1.6|  33|     F|   Analyst|
|   5| Olivia|   1.8|  54|     F|   Teacher|
|   6| Hannah|  1.82|null|     F|      null|
|   7|William|   1.7|  42|     M|  Engineer|
|null|   null|  null|null|  null|      null|
|   8|  Ethan|  1.55|  38|     M|    Doctor|
|   9| Hannah|  1.65|null|     F|    Doctor|
|  10| Xavier|  1.64|  43|  null|    Doctor|
|  11|   null|  null|null|  null|      null|
+----+-------+------+----+------+----------+



Bütün kolonlarda dolu olan(hiç null değeri olmayan) satır sayısı kaç?


In [30]:
df.dropna().count()

12

Bu satırları göster


In [35]:
df.dropna(how = "any").show()

+---+-------+------+---+------+----------+
| Id|   Name|Height|Age|Gender|Profession|
+---+-------+------+---+------+----------+
|  1|   John|  1.79| 28|     M|    Doctor|
|  4| Ashley|   1.6| 33|     F|   Analyst|
|  5| Olivia|   1.8| 54|     F|   Teacher|
|  7|William|   1.7| 42|     M|  Engineer|
|  8|  Ethan|  1.55| 38|     M|    Doctor|
+---+-------+------+---+------+----------+



Eğer satırda bütün değerler null ise o satırı getirme demektir.


In [36]:
df.show()
df.dropna(how = "all").show()

+----+-------+------+----+------+----------+
|  Id|   Name|Height| Age|Gender|Profession|
+----+-------+------+----+------+----------+
|   1|   John|  1.79|  28|     M|    Doctor|
|   2|  Steve|  1.78|  45|     M|      null|
|   3|   Emma|  1.75|null|  null|      null|
|   4| Ashley|   1.6|  33|     F|   Analyst|
|   5| Olivia|   1.8|  54|     F|   Teacher|
|   6| Hannah|  1.82|null|     F|      null|
|   7|William|   1.7|  42|     M|  Engineer|
|null|   null|  null|null|  null|      null|
|   8|  Ethan|  1.55|  38|     M|    Doctor|
|   9| Hannah|  1.65|null|     F|    Doctor|
|  10| Xavier|  1.64|  43|  null|    Doctor|
|  11|   null|  null|null|  null|      null|
+----+-------+------+----+------+----------+

+---+-------+------+----+------+----------+
| Id|   Name|Height| Age|Gender|Profession|
+---+-------+------+----+------+----------+
|  1|   John|  1.79|  28|     M|    Doctor|
|  2|  Steve|  1.78|  45|     M|      null|
|  3|   Emma|  1.75|null|  null|      null|
|  4| Ashley|  

Null değerleri 0 la doldur ama bunu sadece numerik kolonlar için yapar string kolonlar doldurulmaz.


In [11]:
df.fillna(0).show()

+---+-------+------+---+------+----------+
| Id|   Name|Height|Age|Gender|Profession|
+---+-------+------+---+------+----------+
|  1|   John|  1.79| 28|     M|    Doctor|
|  2|  Steve|  1.78| 45|     M|      null|
|  3|   Emma|  1.75|  0|  null|      null|
|  4| Ashley|   1.6| 33|     F|   Analyst|
|  5| Olivia|   1.8| 54|     F|   Teacher|
|  6| Hannah|  1.82|  0|     F|      null|
|  7|William|   1.7| 42|     M|  Engineer|
|  0|   null|   0.0|  0|  null|      null|
|  8|  Ethan|  1.55| 38|     M|    Doctor|
|  9| Hannah|  1.65|  0|     F|    Doctor|
| 10| Xavier|  1.64| 43|  null|    Doctor|
| 11|   null|   0.0|  0|  null|      null|
+---+-------+------+---+------+----------+



string kolonlardaki null değerleri Profession ile doldurur


In [12]:
df.fillna('Profession').show()

+----+----------+------+----+----------+----------+
|  Id|      Name|Height| Age|    Gender|Profession|
+----+----------+------+----+----------+----------+
|   1|      John|  1.79|  28|         M|    Doctor|
|   2|     Steve|  1.78|  45|         M|Profession|
|   3|      Emma|  1.75|null|Profession|Profession|
|   4|    Ashley|   1.6|  33|         F|   Analyst|
|   5|    Olivia|   1.8|  54|         F|   Teacher|
|   6|    Hannah|  1.82|null|         F|Profession|
|   7|   William|   1.7|  42|         M|  Engineer|
|null|Profession|  null|null|Profession|Profession|
|   8|     Ethan|  1.55|  38|         M|    Doctor|
|   9|    Hannah|  1.65|null|         F|    Doctor|
|  10|    Xavier|  1.64|  43|Profession|    Doctor|
|  11|Profession|  null|null|Profession|Profession|
+----+----------+------+----+----------+----------+



Burada sadece Profession kolonunda ki Null değerleri Free ile doldurur


In [13]:
df.fillna('Free', subset=['Profession']).show()

+----+-------+------+----+------+----------+
|  Id|   Name|Height| Age|Gender|Profession|
+----+-------+------+----+------+----------+
|   1|   John|  1.79|  28|     M|    Doctor|
|   2|  Steve|  1.78|  45|     M|      Free|
|   3|   Emma|  1.75|null|  null|      Free|
|   4| Ashley|   1.6|  33|     F|   Analyst|
|   5| Olivia|   1.8|  54|     F|   Teacher|
|   6| Hannah|  1.82|null|     F|      Free|
|   7|William|   1.7|  42|     M|  Engineer|
|null|   null|  null|null|  null|      Free|
|   8|  Ethan|  1.55|  38|     M|    Doctor|
|   9| Hannah|  1.65|null|     F|    Doctor|
|  10| Xavier|  1.64|  43|  null|    Doctor|
|  11|   null|  null|null|  null|      Free|
+----+-------+------+----+------+----------+



Bütün satırlarda 2 ve 2'den fazla null olmayan değer varsa o satırı göster 


In [16]:
ff1=df.dropna(thresh=2)
ff1.show()

+---+-------+------+----+------+----------+
| Id|   Name|Height| Age|Gender|Profession|
+---+-------+------+----+------+----------+
|  1|   John|  1.79|  28|     M|    Doctor|
|  2|  Steve|  1.78|  45|     M|      null|
|  3|   Emma|  1.75|null|  null|      null|
|  4| Ashley|   1.6|  33|     F|   Analyst|
|  5| Olivia|   1.8|  54|     F|   Teacher|
|  6| Hannah|  1.82|null|     F|      null|
|  7|William|   1.7|  42|     M|  Engineer|
|  8|  Ethan|  1.55|  38|     M|    Doctor|
|  9| Hannah|  1.65|null|     F|    Doctor|
| 10| Xavier|  1.64|  43|  null|    Doctor|
+---+-------+------+----+------+----------+



Height, Age, Gender, Profession kolonlarında toplamda 2 ve 2'den az Null değeri olan satırları al


In [17]:
ff2=df.dropna(thresh=2,subset=('Height','Age','Gender','Profession'))
ff2.show()

+---+-------+------+----+------+----------+
| Id|   Name|Height| Age|Gender|Profession|
+---+-------+------+----+------+----------+
|  1|   John|  1.79|  28|     M|    Doctor|
|  2|  Steve|  1.78|  45|     M|      null|
|  4| Ashley|   1.6|  33|     F|   Analyst|
|  5| Olivia|   1.8|  54|     F|   Teacher|
|  6| Hannah|  1.82|null|     F|      null|
|  7|William|   1.7|  42|     M|  Engineer|
|  8|  Ethan|  1.55|  38|     M|    Doctor|
|  9| Hannah|  1.65|null|     F|    Doctor|
| 10| Xavier|  1.64|  43|  null|    Doctor|
+---+-------+------+----+------+----------+



Profession'da null olmayan değerleri Occupied null ise Unknown yaz


In [18]:
from pyspark.sql.functions import when   
yeni=df.withColumn('Profession', when(df.Profession.isNotNull(), 'Occupied').otherwise('Unknown'))
yeni.show()

+----+-------+------+----+------+----------+
|  Id|   Name|Height| Age|Gender|Profession|
+----+-------+------+----+------+----------+
|   1|   John|  1.79|  28|     M|  Occupied|
|   2|  Steve|  1.78|  45|     M|   Unknown|
|   3|   Emma|  1.75|null|  null|   Unknown|
|   4| Ashley|   1.6|  33|     F|  Occupied|
|   5| Olivia|   1.8|  54|     F|  Occupied|
|   6| Hannah|  1.82|null|     F|   Unknown|
|   7|William|   1.7|  42|     M|  Occupied|
|null|   null|  null|null|  null|   Unknown|
|   8|  Ethan|  1.55|  38|     M|  Occupied|
|   9| Hannah|  1.65|null|     F|  Occupied|
|  10| Xavier|  1.64|  43|  null|  Occupied|
|  11|   null|  null|null|  null|   Unknown|
+----+-------+------+----+------+----------+



"Gender", "Profession", "Id", "Name" kolonlarını hariç tutarak , numerik kolonlar için Null değerleri ortalama ile değiştir 


In [19]:
from pyspark.sql.functions import *
def fill_with_mean(this_df, exclude=set()):

    stats = this_df.agg(*(avg(c).alias(c) for c in this_df.columns if c not in exclude))
    return this_df.na.fill(stats.first().asDict())

fill_with_mean(df, ["Gender", "Profession", "Id", "Name"]).show()

+----+-------+------------------+---+------+----------+
|  Id|   Name|            Height|Age|Gender|Profession|
+----+-------+------------------+---+------+----------+
|   1|   John|              1.79| 28|     M|    Doctor|
|   2|  Steve|              1.78| 45|     M|      null|
|   3|   Emma|              1.75| 40|  null|      null|
|   4| Ashley|               1.6| 33|     F|   Analyst|
|   5| Olivia|               1.8| 54|     F|   Teacher|
|   6| Hannah|              1.82| 40|     F|      null|
|   7|William|               1.7| 42|     M|  Engineer|
|null|   null|1.7080000000000002| 40|  null|      null|
|   8|  Ethan|              1.55| 38|     M|    Doctor|
|   9| Hannah|              1.65| 40|     F|    Doctor|
|  10| Xavier|              1.64| 43|  null|    Doctor|
|  11|   null|1.7080000000000002| 40|  null|      null|
+----+-------+------------------+---+------+----------+



Kolon tipi bigint,double,int olan kolonlardaki null değerleri medyan ile değiştir 


In [20]:
num_cols = [col_type[0] for col_type in filter(lambda dtype: dtype[1] in {"bigint", "double", "int"}, df.dtypes)]
### Compute a dict with <col_name, median_value>
median_dict = dict()
for c in num_cols:
   median_dict[c] = df.stat.approxQuantile(c, [0.5], 0.001)[0]
df_imputed=df.na.fill(median_dict)
df_imputed.show()

+---+-------+------+---+------+----------+
| Id|   Name|Height|Age|Gender|Profession|
+---+-------+------+---+------+----------+
|  1|   John|  1.79| 28|     M|    Doctor|
|  2|  Steve|  1.78| 45|     M|      null|
|  3|   Emma|  1.75| 42|  null|      null|
|  4| Ashley|   1.6| 33|     F|   Analyst|
|  5| Olivia|   1.8| 54|     F|   Teacher|
|  6| Hannah|  1.82| 42|     F|      null|
|  7|William|   1.7| 42|     M|  Engineer|
|  6|   null|   1.7| 42|  null|      null|
|  8|  Ethan|  1.55| 38|     M|    Doctor|
|  9| Hannah|  1.65| 42|     F|    Doctor|
| 10| Xavier|  1.64| 43|  null|    Doctor|
| 11|   null|   1.7| 42|  null|      null|
+---+-------+------+---+------+----------+



Null değerler, ortalama hesabında hiçbir şekilde hesaba katılmaz


In [21]:
df.agg(mean("Height")).show()

+------------------+
|       avg(Height)|
+------------------+
|1.7080000000000002|
+------------------+



Height'ı null gelen satırlar hangileridir. Bunun için sql functions'ı yüklememiz gerekiyo


In [22]:
from pyspark.sql import functions as F
df.filter(F.isnull("Height")).show()

+----+----+------+----+------+----------+
|  Id|Name|Height| Age|Gender|Profession|
+----+----+------+----+------+----------+
|null|null|  null|null|  null|      null|
|  11|null|  null|null|  null|      null|
+----+----+------+----+------+----------+



Height'ı null olmayan veya boş olmayan satırlar hangileridir(bizim dataframe'inde boş değilde null değerleri var. Bazı durumlarda boş değerler ilede karşılaşılabilir. 


In [23]:
df.filter((df.Height.isNotNull()) | (df.Height != "")).show()

+---+-------+------+----+------+----------+
| Id|   Name|Height| Age|Gender|Profession|
+---+-------+------+----+------+----------+
|  1|   John|  1.79|  28|     M|    Doctor|
|  2|  Steve|  1.78|  45|     M|      null|
|  3|   Emma|  1.75|null|  null|      null|
|  4| Ashley|   1.6|  33|     F|   Analyst|
|  5| Olivia|   1.8|  54|     F|   Teacher|
|  6| Hannah|  1.82|null|     F|      null|
|  7|William|   1.7|  42|     M|  Engineer|
|  8|  Ethan|  1.55|  38|     M|    Doctor|
|  9| Hannah|  1.65|null|     F|    Doctor|
| 10| Xavier|  1.64|  43|  null|    Doctor|
+---+-------+------+----+------+----------+



boş olmayan değerlerin ortalaması default oralama ile aynı olması gerekir


In [24]:
df.agg(mean("Height")).show()

+------------------+
|       avg(Height)|
+------------------+
|1.7080000000000002|
+------------------+



In [25]:
df.filter((df.Height.isNotNull()) | (df.Height != "")).agg(mean("Height")).show()

+------------------+
|       avg(Height)|
+------------------+
|1.7080000000000002|
+------------------+



Age 30'den büyük kolonların ortalamasını al


In [26]:
df.filter(df['Age'] > 30).agg({"Age": "avg"}).show()

+--------+
|avg(Age)|
+--------+
|    42.5|
+--------+



Profession'a göre yaş ortalamasını ve sayısını,toplamını al(null değerler ayrı değerlenidirilir


In [27]:
import pyspark.sql.functions as f
df.groupBy("Profession").agg(f.mean("Age").alias("Mean"),f.count("Age").alias("Count")
                            ,f.sum("Age").alias("Sum")).show()

+----------+------------------+-----+---+
|Profession|              Mean|Count|Sum|
+----------+------------------+-----+---+
|   Teacher|              54.0|    1| 54|
|      null|              45.0|    1| 45|
|   Analyst|              33.0|    1| 33|
|    Doctor|36.333333333333336|    3|109|
|  Engineer|              42.0|    1| 42|
+----------+------------------+-----+---+



Gendera göre age ortalamasını ve null olmayan professionı say


In [28]:
df.show()
df.groupBy("Gender").agg({'Age':'avg', 'Profession':'count'}).show()

+----+-------+------+----+------+----------+
|  Id|   Name|Height| Age|Gender|Profession|
+----+-------+------+----+------+----------+
|   1|   John|  1.79|  28|     M|    Doctor|
|   2|  Steve|  1.78|  45|     M|      null|
|   3|   Emma|  1.75|null|  null|      null|
|   4| Ashley|   1.6|  33|     F|   Analyst|
|   5| Olivia|   1.8|  54|     F|   Teacher|
|   6| Hannah|  1.82|null|     F|      null|
|   7|William|   1.7|  42|     M|  Engineer|
|null|   null|  null|null|  null|      null|
|   8|  Ethan|  1.55|  38|     M|    Doctor|
|   9| Hannah|  1.65|null|     F|    Doctor|
|  10| Xavier|  1.64|  43|  null|    Doctor|
|  11|   null|  null|null|  null|      null|
+----+-------+------+----+------+----------+

+------+-----------------+--------+
|Gender|count(Profession)|avg(Age)|
+------+-----------------+--------+
|     F|                3|    43.5|
|  null|                1|    43.0|
|     M|                3|   38.25|
+------+-----------------+--------+



40'dan büyük kolonları 1 ile değiştir yoksa 0 ile değiştirelim


In [29]:
from pyspark.sql.functions import when   
cols = df.columns # list of all columns
for col in cols:df= df.withColumn(col, when(df[col]>40,1).otherwise(0))
df.show()

+---+----+------+---+------+----------+
| Id|Name|Height|Age|Gender|Profession|
+---+----+------+---+------+----------+
|  0|   0|     0|  0|     0|         0|
|  0|   0|     0|  1|     0|         0|
|  0|   0|     0|  0|     0|         0|
|  0|   0|     0|  0|     0|         0|
|  0|   0|     0|  1|     0|         0|
|  0|   0|     0|  0|     0|         0|
|  0|   0|     0|  1|     0|         0|
|  0|   0|     0|  0|     0|         0|
|  0|   0|     0|  0|     0|         0|
|  0|   0|     0|  0|     0|         0|
|  0|   0|     0|  1|     0|         0|
|  0|   0|     0|  0|     0|         0|
+---+----+------+---+------+----------+

